Hypotesis: egtm = egt redline - egt

Idea: egt redline may depend on engine hours linearly. I want to plot (egt - etgm) vs engine hours 

In [22]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

In [27]:
import pandas as pd
import matplotlib.pyplot as plt 

dataset = pd.read_csv('./small-sample-BGU-30.csv', parse_dates=['reportts']) \
  .sort_values('reportts')

In [28]:
important_features = [
 'egt', 'naiup', 'nait', 'tec', 'aoc', 'ecyc', 'esn', 'ehrs', 'fdp', 'ps14', 'w14', 'egtb'
]

In [29]:
Y = dataset[['egtm']]

X = dataset.drop(columns=[
    'reportts', 'acnum', 'pos', 'dep', 'arr', 
    'egtm', 'fltdes', 'reportts',
    'dmusw', 'exswpn', 'reason'
]).fillna(0)

X = X.loc[:, ~X.columns.str.contains('stw')]

In [6]:
def train_model(X, y):
  y = Y['egtm']
  x = X[y.notna()]
  y = y.dropna()

  X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=40)

  scaler = StandardScaler()
  scaler.fit(X_train)

  X_train = scaler.transform(X_train)
  X_test = scaler.transform(X_test)
  

  model = LinearRegression(n_jobs=-1)
  model.fit(X_train, y_train)

  predicted = model.predict(X_test)
  preds = pd.DataFrame({'y': y_test, 'pred': predicted})
  mse = mean_squared_error(y_test, predicted, squared=False)
  mae = mean_absolute_error(y_test, predicted)
  
  return mse, mae, model, preds

In [7]:
mse, mae, result_model, pred = train_model(X, Y)
mse

95.22142284012159

In [8]:
mse, mae, result_model, pred = train_model(X[important_features], Y)
mse

2.4396958561090125

Let's do some feature generation

In [9]:
X_aug = X[important_features].copy()
for f in important_features:
  X_aug[f + '_2'] = X_aug[f] ** 2
  for k in important_features:
    if f != k:
      X_aug[f + '_m_' + k] = X_aug[f] * X_aug[k]

In [10]:
mse, mae, result_model, pred = train_model(X_aug, Y)
mse

66.86652931758664

In [11]:
X_aug

,egt,naiup,nait,tec,aoc,ecyc,esn,ehrs,fdp,ps14,...,egtb_m_naiup,egtb_m_nait,egtb_m_tec,egtb_m_aoc,egtb_m_ecyc,egtb_m_esn,egtb_m_ehrs,egtb_m_fdp,egtb_m_ps14,egtb_m_w14
0,800.1,128.8,4.0,13.0,3.6,0,0.0,0,11.3,15.991,...,59776.08,1856.4,6033.3,1670.76,0.0,0.0,0.0,5244.33,7421.4231,538356.0
510,802.4,127.6,3.0,10.0,3.5,0,0.0,0,10.4,15.895,...,59027.76,1387.8,4626.0,1619.10,0.0,0.0,0.0,4811.04,7353.0270,536616.0
1,851.4,129.0,12.0,24.0,3.5,2,0.0,4,12.0,16.026,...,64280.70,5979.6,11959.2,1744.05,996.6,0.0,1993.2,5979.60,7985.7558,587495.7
511,854.0,128.3,12.0,23.0,3.4,2,0.0,4,10.8,15.873,...,63919.06,5978.4,11458.6,1693.88,996.4,0.0,1992.8,5380.56,7907.9286,587377.8
2,851.6,132.3,-3.0,8.0,3.5,3,0.0,6,11.4,16.482,...,62472.06,-1416.6,3777.6,1652.70,1416.6,0.0,2833.2,5383.08,7782.8004,614804.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
507,872.1,119.1,-8.0,2.0,3.6,205,771009.0,957,12.5,16.848,...,60181.23,-4042.4,1010.6,1819.08,103586.5,389590847.7,483572.1,6316.25,8513.2944,677102.0
1018,906.7,109.8,29.0,38.0,3.5,206,771035.0,964,12.6,16.001,...,59237.10,15645.5,20501.0,1888.25,111137.0,415973382.5,520078.0,6797.70,8632.5395,615030.0
508,911.1,112.7,30.0,39.0,3.4,206,771009.0,964,13.1,16.053,...,60531.17,16113.0,20946.9,1826.14,110642.6,414108933.9,517764.4,7036.01,8622.0663,610682.7
509,799.3,118.3,-17.0,-5.0,3.7,279,771009.0,1355,12.8,16.822,...,53731.86,-7721.4,-2271.0,1680.54,126721.8,350192287.8,615441.0,5813.76,7640.5524,606811.2


Equation discovery

In [12]:
# !pip3 install -U pysr -q
# !python3 -m pysr install

In [33]:
important_features = [
    'naiup', 'nait', 'tec', 'aoc', 'ecyc', 'ehrs', 
    'fdp', 'w14', 'odp', 'vb2', 'bbf', 
    'vorrc', 'baf', 't2_peak', 'acct', 'alt_peak', 
    'n1max', 'n1msa', 'shptp', 'egt', 'egt_peak'
]

In [32]:
X[important_features].isna().sum()

naiup        0
nait         0
tec          0
aoc          0
ecyc         0
esn          2
ehrs         0
fdp          0
ps14         7
w14          0
egtb         5
odp          0
vb2          0
bbf          0
vorrc        0
baf          0
t2_peak      0
acct         0
t3           4
alt_peak     0
n1max        0
n1msa        0
oat_peak    10
shptp        0
egt          0
egt_peak     0
dtype: int64

In [34]:
from pysr import PySRRegressor



model = PySRRegressor(
    niterations=1000, 
    binary_operators=["*", "+", "-", "/"],
    unary_operators=[
        "square",
        "inv(x) = 1/x",
    ],
    extra_sympy_mappings={"inv": lambda x: 1 / x},
    loss="loss(prediction, target) = (prediction - target)^2",
)

y = Y['egtm']
x = X[important_features]

model.fit(x, y)

/home/skeptlk/.conda/envs/recsys/lib/python3.9/site-packages/pysr/sr.py:1346: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
/home/skeptlk/.conda/envs/recsys/lib/python3.9/site-packages/pysr/sr.py:1937: UserWarning: Note: you are running with 10 features or more. Genetic algorithms like used in PySR scale poorly with large numbers of features. You should run PySR for more `niterations` to ensure it can find the correct variables, or, alternatively, do a dimensionality reduction beforehand. For example, `X = PCA(n_components=6).fit_transform(X)`, using scikit-learn's `PCA` class, will reduce the number of features to 6 in an interpretable way, as each resultant feature will be a linear combination of the original features. 
  warnings.warn(


Started!

Expressions evaluated per second: 4.880e+05
Head worker occupation: 12.0%
Progress: 1022 / 15000 total iterations (6.813%)
Hall of Fame:
---------------------------------------------------------------------------------------------------
Complexity  Loss       Score     Equation
1           4.474e+01  1.594e+01  y = 28.663
3           1.814e+01  4.513e-01  y = (-95.039 + naiup)
5           1.544e+01  8.065e-02  y = ((naiup - n1max) * 0.85307)
7           1.334e+01  7.319e-02  y = (((naiup - n1max) / w14) * 1032.9)
9           1.315e+01  7.099e-03  y = ((((naiup - n1max) / aoc) + vb2) * 2.9027)
10          1.230e+01  6.645e-02  y = ((square((naiup - n1max) / aoc) * 0.15289) + 13.83)
12          1.219e+01  4.717e-03  y = (((naiup - (n1max - (baf / square(odp)))) / aoc) * 2.8972)
14          1.193e+01  1.062e-02  y = (((naiup - (n1max - square((baf - 1.3443) / odp))) / aoc) ...
                                  * 2.8972)
15          1.193e+01  6.920e-05  y = (((naiup - (n1max - s

PySRRegressor.equations_ = [
	    pick     score                                           equation  \
	0         0.000000                                          28.662617   
	1         0.451307                                (-95.03932 + naiup)   
	2         0.080654                      ((naiup - n1max) * 0.8530907)   
	3         0.077983                ((naiup - n1max) / (shptp / n1msa))   
	4         0.004727        ((((naiup - n1max) / shptp) * n1msa) + vb2)   
	5         0.079886  ((square((naiup - n1max) / aoc) + n1max) * 0.1...   
	6         0.019762  ((square(square((naiup - n1max) / aoc)) * 0.00...   
	7         0.073758  ((square((naiup - n1max) / aoc) + (vorrc / 0.5...   
	8         0.008095  ((vorrc + (square((naiup - n1max) / aoc) + (vo...   
	9         0.000084  ((vorrc + (square((naiup - n1max) / aoc) + (vo...   
	10  >>>>  0.082250  ((((square((naiup - n1max) / aoc) + (vorrc / o...   
	11        0.005533  (((((square((naiup - n1max) / aoc) - odp) + (v...   
	12        0.007267  (((((square((naiup - n1max) / aoc) + ((vorrc *...   
	
	         loss  complexity  
	0   44.737130           1  
	1   18.141293           3  
	2   15.438792           5  
	3   13.209258           7  
	4   13.084960           9  
	5   12.080315          10  
	6   11.843932          11  
	7   11.001787          12  
	8   10.825096          14  
	9   10.824182          15  
	10   9.969521          16  
	11   9.859809          18  
	12   9.717549          20  
]

In [14]:
model.equations_.loc[13].equation

'(((naiup - n1max) * ((n1msa + (alt_peak / ((ehrs + vorrc) - esn))) / (shptp + aoc))) + vb2)'

In [15]:
model.predict(X)

array([43.34428478, 41.96163662, 41.11854404, ..., 17.72685722,
       26.50253059, 21.16224713])

In [16]:
y

0       44.437
510     45.869
1       44.379
511     44.904
2       43.742
         ...  
507     22.152
1018    20.216
508     22.151
509     18.218
1019    17.672
Name: egtm, Length: 1020, dtype: float64

In [17]:
pd.DataFrame({'1': dataset['naiup'] - 95.12702, '2': dataset['egtm']})

,1,2
0,33.67298,44.437
510,32.47298,45.869
1,33.87298,44.379
511,33.17298,44.904
2,37.17298,43.742
...,...,...
507,23.97298,22.152
1018,14.67298,20.216
508,17.57298,22.151
509,23.17298,18.218


In [18]:
test = pd.DataFrame({
    'x1': np.random.random(10) * 2,
    'x2': np.random.random(10) * 5,
})

test['y'] = -10.4 * (test['x1'] ** 2) + 35 * test['x1'] * test['x2'] - 45.5

test


,x1,x2,y
0,1.724011,3.776403,151.458611
1,1.013902,3.301852,60.980260
2,1.287831,1.817820,19.188109
3,1.677003,3.317793,119.989878
4,1.496495,1.304137,-0.483557
5,0.957474,1.877992,7.900227
6,1.792586,3.783662,158.469961
7,0.487391,4.174946,23.248520
8,0.836554,0.648680,-33.785212
9,0.698549,1.411516,-16.064429


In [19]:

# model2 = PySRRegressor(
#     niterations=100,
#     binary_operators=["*", "+", "-", "/"],
#     unary_operators=[
#         "square", "cube"
#         # "cos",
#         # "exp",
#         # "sin",
#         # "inv(x) = 1/x",
#     ],
#     # extra_sympy_mappings={"inv": lambda x: 1 / x},
#     loss="loss(prediction, target) = (prediction - target)^2",
# )

# model2.fit(test[['x1', 'x2']], test['y'])

/home/skeptlk/.conda/envs/recsys/lib/python3.9/site-packages/pysr/sr.py:1346: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(


Started!


PySRRegressor.equations_ = [
	    pick     score                                           equation  \
	0         0.000000                                          49.088055   
	1         0.323818                                           cube(x2)   
	2         1.305684                                     cube(cube(x1))   
	3         0.567034                            (square(x1) * cube(x2))   
	4         0.141571                     square((x2 * -1.9002142) * x1)   
	5         2.071869              (((x2 * 30.648726) * x1) - 48.554134)   
	6         1.582625  ((((x2 * 34.15012) - 21.206758) * x1) - 34.320...   
	7         0.000001  (((((x2 * 34.150963) - 21.207333) * x1) - 34.3...   
	8         0.202189  (((((x2 * 34.231857) - (19.167152 + x1)) * x1)...   
	9         0.547310  ((((((x2 * 35.43658) - x2) - 17.843332) * x1) ...   
	10        0.483268  ((((((x2 * 34.77883) + (-18.018938 - cube(x1))...   
	11  >>>>  0.527231  (((((((x2 * 35.791) - x1) - cube(x1)) + -16.60...   
	12        0.043796  ((((x2 * 34.14693) + ((-12.616798 - (square(x1...   
	
	           loss  complexity  
	0   4444.091300           1  
	1   3214.775400           2  
	2    871.162800           3  
	3    280.271970           5  
	4    243.274260           6  
	5     30.640442           7  
	6      1.293137           9  
	7      1.293132          12  
	8      1.056412          13  
	9      0.611138          14  
	10     0.232476          16  
	11     0.080990          18  
	12     0.074198          20  
]